In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from math import sqrt

from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Activation, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit

In [0]:
#Run Local
#df = pd.read_csv("csv/book_info_complete.csv")
#Run on COLAB
df = pd.read_csv("/content/drive/My Drive/final_project/book_info_complete.csv")

In [0]:
df = df.dropna(axis=1)
df["Publication date"] = df["Publication date"].str[-4:]
df = df.drop("Last updated", axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5816 entries, 0 to 5815
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             5816 non-null   object
 1   description       5816 non-null   object
 2   plot              5816 non-null   object
 3   csm_review        5816 non-null   object
 4   need_to_know      5816 non-null   object
 5   csm_rating        5816 non-null   int64 
 6   Genre             5816 non-null   object
 7   Book type         5816 non-null   object
 8   Publication date  5816 non-null   object
dtypes: int64(1), object(8)
memory usage: 409.1+ KB


In [0]:
df["text"] = df['title'] + " " + df['plot'] + " " + df["description"] + " " + df["csm_review"] + " " + df["need_to_know"] + " " + df["Genre"] + " " + df["Book type"]+ " " + df["Publication date"]

## Create the splits

In [0]:
def splitter(df):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=999)
    for train_index, test_index in split.split(df, df['csm_rating']):
        train_data= df.loc[train_index]
        test_data = df.loc[test_index]
    
    
    return train_data, test_data

In [0]:
train_data, test_data = splitter(df)

In [0]:
x_tr, y_tr = train_data['text'].values, train_data["csm_rating"].values
x_val, y_val = test_data["text"].values, test_data["csm_rating"].values

In [0]:
print(x_tr.shape, y_tr.shape)

(4652,) (4652,)


In [0]:
print(x_val.shape, y_val.shape)

(1164,) (1164,)


## Prepare the Data

In [0]:
#Tokenize the sentences
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(x_tr))

#converting text into integer sequences
x_tr_seq  = tokenizer.texts_to_sequences(x_tr) 
x_val_seq = tokenizer.texts_to_sequences(x_val)

print(len(max(x_tr_seq, key=len)))
max_length = len(max(x_tr_seq, key=len))
print(len(min(x_tr_seq, key=len)))

979
117


In [0]:
#padding to prepare sequences of same length
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=max_length)
x_val_seq = pad_sequences(x_val_seq, maxlen=max_length)
print(len(max(x_tr_seq, key=len)))
print(len(min(x_tr_seq, key=len)))

979
979


In [0]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

45235


## Create embeddings

In [0]:
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

Found 45234 unique tokens.


## Create the Model

In [0]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,input_length=max_length,trainable=True)) 

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='relu')) 

#Add loss function, metrics, optimizer
#optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(optimizer="RMSprop", loss='mean_squared_error', metrics=["mae"]) 

#Print summary of model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 979, 300)          13570500  
_________________________________________________________________
lstm (LSTM)                  (None, 979, 128)          219648    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 13,798,469
Trainable params: 13,798,469
Non-trainable params: 0
_________________________________________________________________
None


## Callbacks

In [0]:
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_mae', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

## Fit the Model

In [0]:
history = model.fit(np.array(x_tr_seq),
                    np.array(y_tr),
                    batch_size=128,
                    epochs=1000,
                    validation_data=(np.array(x_val_seq),np.array(y_val)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/1000
37/37 [==============================] - 15s 403ms/step - loss: 41.3304 - mae: 5.2383 - val_loss: 15.0230 - val_mae: 3.2824 - lr: 0.0010
Epoch 2/1000
37/37 [==============================] - 14s 388ms/step - loss: 15.0801 - mae: 3.2827 - val_loss: 15.0700 - val_mae: 3.2719 - lr: 0.0010
Epoch 3/1000
37/37 [==============================] - 14s 387ms/step - loss: 13.8860 - mae: 3.1478 - val_loss: 11.8234 - val_mae: 2.8864 - lr: 0.0010
Epoch 4/1000
37/37 [==============================] - 14s 390ms/step - loss: 8.6924 - mae: 2.4614 - val_loss: 5.4637 - val_mae: 1.9277 - lr: 0.0010
Epoch 5/1000
37/37 [==============================] - 14s 387ms/step - loss: 4.0125 - mae: 1.6110 - val_loss: 4.1774 - val_mae: 1.5778 - lr: 0.0010
Epoch 6/1000
37/37 [==============================] - 14s 387ms/step - loss: 2.9442 - mae: 1.3345 - val_loss: 3.6961 - val_mae: 1.4578 - lr: 0.0010
Epoch 7/1000
37/37 [==============================] - 14s 386ms/step - loss: 2.3718 - mae: 1.1797 - val_lo

In [0]:
#evaluation 
val_loss, val_mae = model.evaluate(x_val_seq, y_val)

print("The val_mae is %.3f." % val_mae)

37/37 [==============================] - 3s 70ms/step - loss: 2.6056 - mae: 1.2485
The val_mae is 1.249.


In [0]:
model.save('/content/drive/My Drive/final_project/lstm_concat_model') 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/final_project/lstm_concat_model/assets


In [0]:
plt.plot(model.history.history["loss"], label="loss");
plt.plot(model.history.history["val_loss"], label="val_loss");
plt.legend();
plt.show();
plt.close();

plt.plot(model.history.history["mae"], label="mae");
plt.plot(model.history.history["val_mae"], label="val_mae");
plt.legend();

KeyError: ignored

## [Use Transfer Learning](https://www.analyticsvidhya.com/blog/2020/03/pretrained-word-embeddings-nlp/)

In [0]:
# load the whole embedding into memory
embeddings_index = dict()

with open("/content/drive/My Drive/Colab Notebooks/glove.6B/glove.6B.300d.txt") as f:

    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((size_of_vocabulary, 300))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
model2=Sequential()

#embedding layer
model2.add(Embedding(size_of_vocabulary,
                    300,
                    weights=[embedding_matrix],
                    input_length=max_length,
                    trainable=False)) 

#lstm layer
model2.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model2.add(GlobalMaxPooling1D())

#Dense Layer
model2.add(Dense(64,activation='relu')) 
model2.add(Dense(1,activation='relu')) 

model2.compile(optimizer="RMSprop", loss='mean_squared_error', metrics=["mae"]) 

#Print summary of model
print(model2.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 979, 300)          13570500  
_________________________________________________________________
lstm_3 (LSTM)                (None, 979, 128)          219648    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 13,798,469
Trainable params: 227,969
Non-trainable params: 13,570,500
_________________________________________________________________
None


In [0]:
history = model2.fit(np.array(x_tr_seq),
                    np.array(y_tr),
                    batch_size=128,
                    epochs=1000,
                    validation_data=(np.array(x_val_seq),np.array(y_val)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/1000
37/37 [==============================] - 11s 298ms/step - loss: 16.2079 - mae: 3.2653 - val_loss: 7.7052 - val_mae: 2.3558 - lr: 0.0010
Epoch 2/1000
37/37 [==============================] - 11s 286ms/step - loss: 5.4354 - mae: 1.8770 - val_loss: 3.3299 - val_mae: 1.3977 - lr: 0.0010
Epoch 3/1000
37/37 [==============================] - 11s 286ms/step - loss: 3.4333 - mae: 1.4625 - val_loss: 4.1777 - val_mae: 1.5951 - lr: 0.0010
Epoch 4/1000
37/37 [==============================] - 11s 285ms/step - loss: 2.9367 - mae: 1.3500 - val_loss: 2.9738 - val_mae: 1.3267 - lr: 0.0010
Epoch 5/1000
37/37 [==============================] - 11s 285ms/step - loss: 2.4880 - mae: 1.2463 - val_loss: 3.1115 - val_mae: 1.3462 - lr: 0.0010
Epoch 6/1000
37/37 [==============================] - 11s 285ms/step - loss: 2.4849 - mae: 1.2398 - val_loss: 2.5036 - val_mae: 1.2001 - lr: 0.0010
Epoch 7/1000
37/37 [==============================] - 11s 285ms/step - loss: 2.1737 - mae: 1.1566 - val_loss: 2

In [0]:
#evaluation 
_, val_mae = model2.evaluate(x_val_seq, y_val)

print("The val_mae is %.3f." % val_mae)

37/37 [==============================] - 3s 69ms/step - loss: 1.8384 - mae: 1.0521
The val_mae is 1.052.


In [0]:
model.save('/content/drive/My Drive/final_project/lstm_concat_transfer_model') 

INFO:tensorflow:Assets written to: /content/drive/My Drive/final_project/lstm_concat_transfer_model/assets


In [0]:
plt.plot(model2.history.history["loss"], label="loss");
plt.plot(model2.history.history["val_loss"], label="val_loss");
plt.legend();
plt.show();
plt.close();

plt.plot(model2.history.history["mae"], label="mae");
plt.plot(model2.history.history["val_mae"], label="val_mae");
plt.legend();

KeyError: ignored

In [0]:
a = model.predict(x_val_seq)
b = model2.predict(x_val_seq)
#a = min_max_scaler_target.inverse_transform(a)
#b = min_max_scaler_target.inverse_transform(b)

csm_lstm_predictions_df = pd.DataFrame({"concat_custom": list(a), "concat_transfer": list(b)},
                              columns = ["concat_custom", "concat_transfer"], 
                              index=test_data.index)

csm_lstm_predictions_df.to_csv('/content/drive/My Drive/final_project/lstm_concat.csv', index="False")
